# Python dilinde Yapay Sinir Ağları (YSA) Modellemesi

Mustafa Erden
Ses Ar-Ge Ekip Lideri, Sestek


Problem:

Chatbot'a şöyle içerik geldiğinde bunu nasıl anlaşılır formata dönüştürürüz?

"faturabilgisi öğrenmek istiyorum"



## İleri beslemeli YSA

![feedforward_nnet](feedforward_nnet.png)

ref: https://cdn-images-1.medium.com/max/800/1*pbk9xtz7WbBwYPVATdl9Vw.png

Gereklilikler:
* [miniconda](https://docs.conda.io/en/latest/miniconda.html) kurulur.
* Miniconda kurulumu sırasında Path'e ekle opsiyonu seçilir.
* cmd
```console
conda update conda
conda install python=3.6
pip install numpy
pip install cntk / pip install cntk-gpu
pip install tensorboard
pip install tensorflow
```

```python

ali okula gitti

 	 	 	 	 	{a}	l	i	o	k	u	1
 	 	 	 	a	{l}	i	o	k	u	l	1
 	 	 	a	l	{i}	o	k	u	l	a	0
 	a	l	i	 	{o}	k	u	l	a	g	1
a	l	i	 	o	{k}	u	l	a	g	i	1
l	i	 	o	k	{u}	l	a	g	i	t	1
i	 	o	k	u	{l}	a	g	i	t	t	1
 	o	k	u	l	{a}	g	i	t	t	i	0
k	u	l	a	 	{g}	i	t	t	i	h	1
u	l	a	 	g	{i}	t	t	i	h	a	1
l	a	 	g	i	{t}	t	i	h	a	s	1
a	 	g	i	t	{t}	i	h	a	s	a	1
 	g	i	t	t	{i}	h	a	s	a	n	0
```

In [1]:
import io
import re
import numpy as np


tr_upper = list(" ABCÇDEFGĞHIİJKLMNOÖPRSŞTUÜVYZ")
tr_lower = list(" abcçdefgğhıijklmnoöprsştuüvyz")
left_context = 9
right_context = 9

In [2]:
def line_to_seq(line):
    seq = []
    for ch in line:
        if ch in tr_lower:
            seq.append(tr_lower.index(ch))
        elif ch in tr_upper:
            seq.append(tr_upper.index(ch))
        else:
            seq.append(tr_lower.index(' '))
    return seq

In [3]:
def add_context(items, index):
        items_count = len(items)

        go_back = []
        found_count = 0
        cursor = index
        while True:
            if left_context == 0:
                break
            target = 0
            cursor -= 1
            if cursor >= 0:
                target = items[cursor]

            go_back.append(target)
            found_count += 1
            if found_count >= left_context:
                break

        go_forward = []
        found_count = 0
        cursor = index
        while True:
            if right_context == 0:
                break
            target = 0
            cursor += 1
            if cursor < items_count:
                target = items[cursor]

            if cursor < items_count and target == 0:
                continue

            go_forward.append(target)
            found_count += 1
            if found_count >= right_context:
                break

        prev_items = list(reversed(go_back))

        return prev_items + [items[index]] + go_forward

In [4]:
def prepare_ctf_data(data_file, ctf_file):
    with io.open(data_file, mode='rt', encoding='utf-8') as fr:
        lines = fr.read().splitlines()

    zero_sample_count = 0
    one_sample_count = 0
    with open(ctf_file, mode="wt", encoding="utf-8") as fw:
        for line in lines:
            normalized_line = re.sub(r'\s+', ' ', line)
            normalized_line = normalized_line.strip() + ' '
            seq = line_to_seq(normalized_line)

            for i in range(len(seq) - 1):
                next_item = seq[i + 1]
                label = 0 if (next_item == 0) else 1
                feat = np.array(add_context(seq, i)) / len(tr_upper)

                if label == 1:
                    one_sample_count += 1
                else:
                    zero_sample_count += 1

                label_str = "0 1" if label == 1 else "1 0"
                feature_str = " ".join(["{:.6f}".format(k) for k in feat])
                line = "|labels {} |features {}\n".format(label_str, feature_str)
                fw.write(line)

    print("one_sample_count: {} zero_sample_count: {}".format(one_sample_count, zero_sample_count))


In [5]:
prepare_ctf_data("validation.txt", "validation.ctf")

one_sample_count: 42648 zero_sample_count: 8781


In [6]:
prepare_ctf_data("train.txt", "train.ctf")

one_sample_count: 1501143 zero_sample_count: 324323


In [8]:
prepare_ctf_data("train_subset.txt", "train_subset.ctf")

one_sample_count: 68542 zero_sample_count: 13306
